In [ ]:
from pyspark.ml import Pipeline

import sparknlp

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.training import CoNLL


import pandas as pd
import os

# to use GPU 
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

In [ ]:
#Converts our regular CONLL files to SparkNLP's CONLL format
def convert_format(inputpath, outputpath):
    # create the training file
    with open(inputpath) as fp:
        text = fp.readlines()
    text = "".join(text[1:]).split("\n\n") 
    df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], 
                      columns=["Token","Pos","Pos_special","Entity_label"])
    
    # creating the training data
    conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
    for t in range(len(text)):    
        df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
        tokens = df.Token.tolist()
        pos_labels = df.Pos.tolist()
        entity_labels = df.Entity_label.tolist()
        for token, pos, label in zip(tokens,pos_labels,entity_labels):
            conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
        conll_lines += "\n"
        
    with open(outputpath,"w") as fp:
        for line in conll_lines:
            fp.write(line)
    
    print("Done")

In [ ]:
training_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/train/sample.train')
dev_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/dev/sample.train')
test_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio-splits/test/sample.train')


In [ ]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en').setInputCols(["sentence",'token']).setOutputCol("bert").setCaseSensitive(True)#.setMaxSentenceLength(512)
# transforming the training data into embeddings and saving it as parquet files
readyTrainingData = bert.transform(training_data)
readyTrainingData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_train")
# transforming the development data into embeddings and saving it as parquet files
readyDevData = bert.transform(dev_data)

readyDevData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_dev")

# transforming the test data into embeddings and saving it as parquet files
readyTestData = bert.transform(test_data)

readyTestData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_test")

readyTrainingData = spark.read.parquet("/tmp/conll2003/bert_train")
readyDevData = spark.read.parquet("/tmp/conll2003/bert_dev")
readyTestData = spark.read.parquet("/tmp/conll2003/bert_test")



In [ ]:
# initialize NER tagger
nerTagger = NerDLApproach()\
.setInputCols(["sentence", "token", "bert"])\
.setLabelColumn("label")\
.setOutputCol("ner")\
.setMaxEpochs(10)\
.setBatchSize(4)\
.setEnableMemoryOptimizer(True)\
.setRandomSeed(0)\
.setVerbose(1)\
.setValidationSplit(0.2)\
.setEvaluationLogExtended(True)\
.setEnableOutputLogs(True)\
.setIncludeConfidence(True)\
.setTestDataset("/tmp/conll2003/bert_dev")

In [ ]:
# train the model
%time myNerModel = nerTagger.fit(readyTrainingData)

In [ ]:
def get_results(readyTestData):
    results = myNerModel.transform(readyTestData).select("sentence","token","label","ner").collect()    
    #test_data.show()
    
    # to find exceptions where no. of labels does not match no. of ners detected
    count = 0
    indices = []
    for i,row in enumerate(results):
        if len(row['label']) != len(row['ner']):
            count += 1
            indices.append(i)

    print(count)
    print(indices)

    exclusion_list = [results[t] for t in indices]
    results = [results[i] for i in range(len(results)) if i not in indices]
    
    tokens = []
    labels = []
    ners = []

    for row in results:
        tokens.append([t['result'] for t in row['token']])
        labels.append([t['result'] for t in row['label']])
        ners.append([t['result'] for t in row['ner']])

    from seqeval.metrics import accuracy_score, f1_score, classification_report
    #print(accuracy_score(labels,ners))
    #print(f1_score(labels,ners))

    print(classification_report(labels,ners, zero_division=1,digits=6))
